In [1]:
import os
import h5py
from nltk.util import usage
import numpy as np
import datetime
import time
import warnings

from featureExtraction.tf.tf import *
from featureExtraction.tfidf.tfidf import *
from featureExtraction.fasttext.fasttext import *
from featureExtraction.lexicon.lexicon import *
from featureExtraction.orthography.orthography import *
from featureExtraction.pos.postag import *
from featureExtraction.tf.tf import *
from featureExtraction.word2vec.word2vec import *
from featureExtraction.ngram.ngram import *

from featureSelection.kFold import *

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
templated = [x for x in os.listdir("./Dataset/templated")]
templated.sort()
clean = [x for x in os.listdir("./Dataset/clean")]
clean.sort()

DATA_TEMPLATED = ["./Dataset/templated/" + x for x in os.listdir("./Dataset/templated")]
DATA_TEMPLATED.sort()
len_templated = len(DATA_TEMPLATED)
DATA_TEMPLATED1 = [DATA_TEMPLATED[0]]
DATA_CLEAN = ["./Dataset/clean/" + x for x in os.listdir("./Dataset/clean")]
DATA_CLEAN.sort()
len_clean = len(DATA_CLEAN)

In [3]:
warnings.filterwarnings("ignore")
score_list = []
# for i in DATA_TEMPLATED1:
count = 1
fail = 0
for i in DATA_TEMPLATED:
    print("Running {}/{}".format(count, len_templated), end=" ")
    try:
        # timer mulai
        result_fe, label = run_tfidf_tweet(i)

        # create dataset
        # X, y = make_classification(n_samples=len(result_fe), n_features=len(result_fe[0]), random_state=1)
        # prepare the cross-validation procedure
        cv = KFold(n_splits=20, random_state=1, shuffle=True)
        x_train, x_test, y_train, y_test = train_test_split(result_fe, label)
              
        logRes = LogisticRegression(random_state=1, max_iter=1000)
        logRes.fit(x_train, y_train)
        predictions = logRes.predict(x_test)
        
        scores = cross_val_score(logRes, result_fe, label, scoring='f1_micro', cv=cv, n_jobs=-1)
        print('F1-score: %.3f (%.3f)' % (mean(scores), std(scores)))
        score_list.append(mean(scores))
        count += 1
        # timer selesai
    except Exception as e:
        count += 1
        fail += 1
        print(i, ": ", e)
        pass

print((sum(score_list)/len(score_list)))
print("Failed: {}".format(fail))

Running 1/24 F1-score: 0.945 (0.067)
Running 2/24 F1-score: 0.828 (0.097)
Running 3/24 F1-score: 0.988 (0.006)
Running 4/24 F1-score: 0.874 (0.054)
Running 5/24 F1-score: 0.812 (0.033)
Running 6/24 F1-score: 0.965 (0.010)
Running 7/24 F1-score: 0.875 (0.078)
Running 8/24 F1-score: 0.828 (0.080)
Running 9/24 F1-score: 0.823 (0.106)
Running 10/24 F1-score: 0.850 (0.116)
Running 11/24 F1-score: 0.759 (0.057)
Running 12/24 F1-score: 0.910 (0.083)
Running 13/24 F1-score: 0.747 (0.058)
Running 14/24 F1-score: 0.706 (0.098)
Running 15/24 F1-score: 0.840 (0.150)
Running 16/24 F1-score: 0.791 (0.043)
Running 17/24 F1-score: 0.841 (0.021)
Running 18/24 F1-score: 0.606 (0.068)
Running 19/24 F1-score: 0.717 (0.095)
Running 20/24 F1-score: 0.908 (0.039)
Running 21/24 F1-score: 0.647 (0.031)
Running 22/24 F1-score: 0.647 (0.031)
Running 23/24 F1-score: 0.625 (0.049)
Running 24/24 F1-score: 0.763 (0.074)
0.8038641050919394
Failed: 0
